<a href="https://colab.research.google.com/github/iiVSX/SWE3032_41_team10/blob/preprocessing/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

In [4]:
import pandas as pd
dataset = pd.read_excel('Dataset.xlsx')

happy_data = dataset[dataset['감정'] == '행복']
sad_data = dataset[dataset['감정'] == '슬픔']
fear_data = dataset[dataset['감정'] == '공포']
neutral_data = dataset[dataset['감정'] == '중립']
aversion_data = dataset[dataset['감정'] == '혐오']
surprise_data = dataset[dataset['감정'] == '놀람']

print(len(happy_data))
print(len(sad_data))
print(len(fear_data))
print(len(neutral_data))
print(len(aversion_data))
print(len(surprise_data))

11615
21239
9697
51900
10308
12519


In [5]:
from sklearn.utils import resample

minority_data_size = len(fear_data)

happy_data_resampled = resample(happy_data, replace=False, n_samples=minority_data_size, random_state=42)
sad_data_resampled = resample(sad_data, replace=False, n_samples=minority_data_size, random_state=42)
neutral_data_resampled = resample(neutral_data, replace=False, n_samples=minority_data_size, random_state=42)
aversion_data_resampled = resample(aversion_data, replace=False, n_samples=minority_data_size, random_state=42)
surprise_data_resampled = resample(surprise_data, replace=False, n_samples=minority_data_size, random_state=42)

balanced_data = pd.concat([happy_data_resampled, sad_data_resampled, fear_data, neutral_data_resampled, aversion_data_resampled, surprise_data_resampled])

print(dataset)
print(balanced_data)

                                                  문장  감정
0               아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!  분노
1       그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.  혐오
2                                            손님 왔어요.  중립
3                                            손님? 누구?  중립
4                                     몰라요. 팀장님 친구래요.  중립
...                                              ...  ..
138207                                 나 엘리베이터에 갇혔어.  공포
138208                             하지만 기분이 나쁜 걸 어떡해?  분노
138209                             자취방 엘리베이턴데 정전인가봐.  공포
138210                               나 드디어 프로젝트 끝났어!  행복
138211                                    걱정해줘서 고마워.  중립

[138212 rows x 2 columns]
                                                       문장  감정
88040                      끝까지 한번 가보자끝까지해 ~~!!!!!!ㅋㅋㅋㅋㅋㅋㅋ  행복
121949      사람들한테 내가 이벤트에 당첨 돼서 갖고 싶었던 향수를 선물로 받았다고 자랑했어.  행복
122817  내가 10km 마라톤 대회에 출전했는데 이전의 내 기록을 훨씬 갱신했어. 너무 기분...  행복
85385                                    

In [6]:
!pip install transformers seqeval[gpu]
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=de6e02d347428ab22ae1104ed53351f64cb820212e36fe1f1942c68a18121059
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [37]:
# GPU 사용
device = torch.device("cuda")

In [38]:
from transformers import ElectraModel, ElectraTokenizer

model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
class NSMCDataset(Dataset):
  
  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [40]:
#TODO : dataset -> train + test
train_dataset = NSMCDataset(balanced_data, tokenizer)
test_dataset = NSMCDataset(balanced_data, tokenizer)

           문장     감정
count   58182  58182
unique  53833      6
top        네?     행복
freq      188   9697
           문장     감정
count   58182  58182
unique  53833      6
top        네?     행복
freq      188   9697


In [46]:
#model tokenization test
text, attention_mask, y = train_dataset[0]
model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.2004, -0.1811, -0.3011,  ...,  0.0361, -0.2400, -0.7042],
         [ 0.1555, -0.4118, -0.3011,  ..., -0.1434, -0.5062,  0.1731],
         [-0.2247,  0.1194, -0.3759,  ..., -0.0342, -0.4877,  0.0607],
         ...,
         [-0.7510, -0.3428, -0.3082,  ..., -0.0295, -0.0084, -0.0384],
         [-0.4467, -0.7305, -0.4086,  ...,  0.1170,  0.0113,  0.1975],
         [-0.2585, -0.5607, -0.5306,  ..., -0.2245,  0.1046,  0.0844]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [42]:
#execute when previous model exists
model.load_state_dict(torch.load("model.pt"))

FileNotFoundError: ignored

In [19]:
# model's layer
model

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(35000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0-11): 12 x ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0

In [44]:
#hyperparameters
epochs = 5
batch_size = 16

In [21]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
# TODO : current label (text) -> number
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/3637 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
losses, accuracies

In [47]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/3637 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "model.pt")